# Reading input data and building the SFS

The first step when running delimitpy is to read the input alignments and use them to construct a site frequency spectrum (SFS).

In [3]:
from delimitpy import parse_input
from delimitpy import process_empirical